# 04: Point Localization

*Authors: Lars Nitzschke, Prof. Dr. Kevin Buchin*

This notebook serves as supplementary learning material for the course **Geometric Algorithms**.
It showcases and explains implementations of algorithms presented in the corresponding lecture, and elaborates on some practical considerations concerning their use.
Furthermore, it offers interactive visualisations and animations.

## Table of Contents

1. Introduction  
2. Setup  
3. Algorithms  
    3.1. Slab Decomposition  
    3.2. Vertical Decomposition
4. References  

## 1. Introduction

The **point localization problem** was stated in the lecture as follows: Preprocess a planar subdivision such that for any query point q, the face of the subdivision containing q can be given efficiently.

For the **one dimensional** problem it was shown, that the problem can be solved efficiently using a balanced binary search tree.

[Image with example ?]

## 2. Setup

First let's do some setup. This is not very interesting, so you can skip to Section 3 if you want.

We now import everything we'll need throughout this notebook from external sources, including our module for generic data structures, our module for geometric primitives and operations as well as our module for visualisation purposes. 
These modules are explained in [notebook no. 00](./00-Basics.ipynb).

In [ ]:
# Python standard library imports
import math
from typing import Any, Optional
from itertools import combinations, chain

# Data structure, geometry and visualisation module imports
from modules.data_structures import BinaryTree, BinaryTreeDict, Comparator, ComparisonResult as CR, DoublyConnectedSimplePolygon, HalfEdge, DoublyConnectedEdgeList, PointLocation, Face, VDLineSegment, AnimationBinaryTreeDict
from modules.geometry import Point, PointReference, PointSequence, LineSegment, Orientation as ORT, EPSILON, Rectangle
from modules.visualisation import VisualisationTool, PointLocationInstance, SlabDecompositionMode, VerticalExtensionMode, PointLocationMode, PolygonMode

Additionally, we create an object for our visualisation tool and register a few example instances.

In [ ]:
visualisation_tool = VisualisationTool(400, 400, PointLocationInstance(drawing_epsilon=10))
canvas_size = min(visualisation_tool.width, visualisation_tool.height)

c = 0.5 * canvas_size
r = 0.75 * c
s = c - r
t = c + r
u = (t - s) / 30

points = [Point(80, 140), Point(260, 130), Point(340, 200), Point(170, 280), Point(50, 220), Point(200, 60)]
edges = [(0,1), (1,2), (2,3), (4,1), (5,1)]

dcel = DoublyConnectedEdgeList(points, edges)
visualisation_tool.register_example_instance("Testing", dcel)

points = [Point(s + 20*u, t),        Point(s + 26*u, t -  6*u), Point(s + 30*u, t -  8*u), Point(s + 28*u, t - 10*u),
          Point(s + 18*u, t - 12*u), Point(s + 29*u, s + 14*u), Point(s + 27*u, s +  6*u), Point(s + 22*u, s +  1*u),
          Point(s + 14*u, s +  4*u), Point(s + 10*u, s),        Point(s +  2*u, s +  5*u), Point(s + 12*u, s + 12*u),
          Point(s +  4*u, s + 13*u), Point(s +  1*u, s + 16*u), Point(s +  3*u, t - 13*u), Point(s +  5*u, t -  4*u),
          Point(s + 10*u, t -  1*u), Point(s + 14*u, t - 15*u), Point(s + 18*u, s +  8*u)]
          
edges = [(0,1), (1,2), (2,3), (3,4), (4,5), (5,6), (6,7), (7,8), (8,9), (9,10), (10,11), (11,12), (12,13), (13,14),
         (14,15), (15,16), (16,0), (1,3), (4,16), (4,17), (12,17), (14,17), (5,11), (6,18), (10,18)]

# For adding an inner component:
#points.extend([Point(s + 16*u, t -  6*u), Point(s + 20*u, t -  2*u), Point(s + 22*u, t -  8*u)])
#edges.extend([(19,20), (20, 21), (18,21)])

dcel = DoublyConnectedEdgeList(points, edges)
visualisation_tool.register_example_instance("Course Example", dcel)

visualisation_tool.display()

In [ ]:
# visualisation_tool = VisualisationTool(400, 400, PointLocationInstance())
# pl = PointLocation(Rectangle(Point(0, 0), Point(400, 400)), dcel)
# visualisation_tool.register_example_instance("Testing example", pl)

def build_vertical_decomposition(pl: PointLocation) -> PointSequence:
    pl.build_vertical_decomposition(pl.dcel_prepocessing(pl._dcel), random_seed = 42)
    return pl._vertical_decomposition._point_sequence


visualisation_tool.register_algorithm("Build Vert. Decomp.", build_vertical_decomposition, VerticalExtensionMode())

# visualisation_tool._example_buttons[0].click()
# visualisation_tool._algorithm_buttons[0].click()
# visualisation_tool._random_button.click()

In [ ]:
def point_location(pl: PointLocation) -> PointSequence:
    point = pl._dcel._last_added_vertex.point
    point_edges = pl._dcel.find_edges_of_vertex(pl._dcel._last_added_vertex)
    if len(point_edges) > 1 or point_edges[0].origin != point_edges[0].destination:
        raise ValueError(f"Point Location needs to search for a single point not connected to any other point.")
    dcel_face, ps = pl._search_structure.query(point)
    return ps        

def preprocessing(pl: PointLocation) -> PointSequence:
    pl.build_vertical_decomposition(PointLocation.dcel_prepocessing(pl._dcel), random_seed = 42)

visualisation_tool.register_algorithm("Point Location", point_location, PointLocationMode(), preprocessing=preprocessing)

visualisation_tool.display()

In [ ]:
#visualisation_tool._random_button.click()
#visualisation_tool._algorithm_buttons[0].click()
#visualisation_tool._algorithm_buttons[1].click()

#visualisation_tool.handle_click_on_multi_canvas(190,  90)
#visualisation_tool.handle_click_on_multi_canvas(220, 110)
#visualisation_tool.handle_click_on_multi_canvas(220, 110)
#visualisation_tool.handle_click_on_multi_canvas(140, 100)
#visualisation_tool.handle_click_on_multi_canvas(140, 100)
#visualisation_tool.handle_click_on_multi_canvas(190,  90)
#visualisation_tool.handle_click_on_multi_canvas(250,  80)

In [ ]:
# TODO: Animation of DCEL drawings

# TODO: DCEL check for crossings and raise exception or add as segment not as an edge
# TODO: Clicking the canvas sometimes does not work

In [ ]:
#print("A")
#visualisation_tool.handle_click_on_multi_canvas(200, 200)
#print("B")
#visualisation_tool.handle_click_on_multi_canvas(200, 200)
#
#
#print("C")
#visualisation_tool.handle_click_on_multi_canvas(100, 200)
#print("D")
#visualisation_tool.handle_click_on_multi_canvas(100, 200)
#
#
#print("E")
#visualisation_tool.handle_click_on_multi_canvas(50, 50)
#print("F")
#visualisation_tool.handle_click_on_multi_canvas(50, 50)
#
#
#print("G")
#visualisation_tool.handle_click_on_multi_canvas(200, 200)

## 3. Algorithms

### 3.1 Slab Decomposition

In [ ]:
class PointXComparator(Comparator[Point]):
    def compare(self, item: Any, key: Point) -> CR:
        if not isinstance(item, Point):
            raise TypeError("This comparator can only compared points to points")
        elif abs(item.x - key.x) < EPSILON:
            return CR.MATCH
        elif item.x - key.x > EPSILON:
            return CR.AFTER
        else:
            return CR.BEFORE
        
class PointToLineSegmentComparator(Comparator[LineSegment]):
    def compare(self, item: Any, key: LineSegment) -> CR:
        if not isinstance(item, Point):
            if not isinstance(item, LineSegment):
                raise TypeError("This comparator can only compare either points or line segments to line segments")
            elif abs(item.y_from_x(key.left.x) - key.left.y) < EPSILON and abs(item.y_from_x(key.right.x) - key.right.y) < EPSILON:
                return CR.MATCH
            elif item.left.y - key.left.y > EPSILON \
                or abs(item.y_from_x(key.left.x) - key.left.y) < EPSILON and item.y_from_x(key.right.x) - key.right.y > EPSILON:
                return CR.AFTER
            else:
                return CR.BEFORE
                
        elif abs(item.y - key.y_from_x(item.x)) < EPSILON:
            return CR.MATCH
        elif item.y - key.y_from_x(item.x) > EPSILON:
            return CR.AFTER
        else:
            return CR.BEFORE

In [ ]:
class SlabDecomposition:
    def __init__(self, dcel: DoublyConnectedEdgeList, bounding_box: Rectangle = Rectangle(Point(0,0), Point(400, 400)), epsilon = EPSILON):
        self._x_comparator = PointXComparator()
        self._y_comparator = PointToLineSegmentComparator()
        self._tree: AnimationBinaryTreeDict[Point, AnimationBinaryTreeDict[LineSegment, Face]] = AnimationBinaryTreeDict(self._x_comparator, lambda p: p)
        self._ubounded_face = dcel.outer_face
        self._bounding_box = bounding_box
        self._point_sequence = PointSequence()

        sorted_points = sorted(dcel.vertices, key = lambda vertex: vertex.point.x)
        currently_crossing_edges: list[HalfEdge] = []
        
        iterator = iter(sorted_points)
        next_vertex = next(iterator, None)

        while next_vertex is not None:
            vertex = next_vertex
            next_vertex = next(iterator, None)

            if vertex.outgoing_edges() == []:  # Single vertex
                continue

            currently_crossing_edges = list(filter(lambda edge: edge.right.x - vertex.x > epsilon, currently_crossing_edges))

            currently_crossing_edges.extend(filter(lambda edge: edge.right.x - vertex.x > epsilon, vertex.outgoing_edges()))

            if next_vertex is not None and abs(vertex.x - next_vertex.x) < epsilon:  # Same x-coordinate
                self._point_sequence.animate(vertex.point)
                continue

            subtree = AnimationBinaryTreeDict(self._y_comparator, lambda ls: PointReference([ls.left, ls.right], 0))
            for edge in currently_crossing_edges:
                subtree.insert(LineSegment(edge.left, edge.right), edge.incident_face)

            self._tree.insert(vertex.point, subtree)
            self._point_sequence.append(PointReference([vertex.point, Point(vertex.point.x, self._bounding_box.lower), Point(vertex.point.x, self._bounding_box.upper)], 0))

    def query(self, point: Point) -> tuple[Face, PointSequence]:
        ps = PointSequence()
        ps.append(point)
        # Search in x direction for the correct slab
        x_pred, x_ps = self._tree.search_predecessor(point)
        ps = ps + x_ps
        if x_pred is None:  # The search point is in the unbounded slab on the left side
            ps.clear()
            ps.append(point)
            return self._ubounded_face, ps
        else:
            y_tree = x_pred[1]
            # Search inside the slab for the correct face
            y_pred, y_ps = y_tree.search_predecessor(point)
            ps = ps + y_ps
            if y_pred is None:  # The search point is in the unbounded slab on the right side
                ps.clear()
                ps.append(point)
                return self._ubounded_face, ps
            dcel_face = y_pred[1]
            
            ps.clear()
            ps.append(point)
            for point in dcel_face.outer_points():
                ps.append(point)
            return dcel_face, ps


def build_slab_decompostion(pl: PointLocation) -> PointSequence:
    sd = SlabDecomposition(pl._dcel)
    return sd._point_sequence

visualisation_tool.register_algorithm("Build Slab Decomp.", build_slab_decompostion, VerticalExtensionMode())

def slab_point_location(pl: PointLocation) -> PointSequence:
    point = pl._dcel._last_added_vertex.point
    point_edges = pl._dcel.find_edges_of_vertex(pl._dcel._last_added_vertex)
    if len(point_edges) > 1 or point_edges[0].origin != point_edges[0].destination:
        raise ValueError(f"Point Location needs to search for a single point not connected to any other point.")
    face, ps = pl._search_structure.query(point)
    return ps

def slab_preprocessing(pl: PointLocation) -> PointSequence:
    pl._search_structure = SlabDecomposition(pl._dcel)

visualisation_tool.register_algorithm("Slab PL", slab_point_location, PointLocationMode(), preprocessing=slab_preprocessing)


visualisation_tool.display()

### 3.2 Vertical Decomposition